In [40]:
import numpy as np
import matplotlib.pyplot as plt
import sys

In [41]:
class Neuron:
    """Neuron in neural network
    
    __init__(num_in,bias)
        num_in = number of features to input to neuron (creates the number of weights on the neuron)
        bias = True if a bias term should be included for w*x+b 
               False if no bias term should be included for w*x+b
               default=True
           

    forward(x)
        x = input to the neuron for w*x + b
    
    
    backward(gradient_l)
        gradient_l = the upstream gradient for backpropogation
        gradient_x = the local gradient of input x
        gradient_w = the local gradient of the neuron weights w
        
        return the local gradients * the upstream gradient
    """
    
    def __init__(self,num_in,bias=True):
        self.gradient_x = None
        self.gradient_w = None
        
        if bias:
            num_in = num_in + 1
            
        self.weights = np.random.uniform(-1,1,size=(num_in,1))
        self.x = None
        self.bias=bias
    
    def forward(self,x):
        if self.bias == True:
            x = np.concatenate((x,[[1]]),axis=1)
            
        self.x = x
        return np.matmul(x,self.weights)
                                
    def backward(self,gradient_l=1):
        if self.x is not None:
            self.gradient_x = self.weights.transpose()*gradient_l
            self.gradient_w = self.x.transpose()*gradient_l
            return self.gradient_x,self.gradient_w
        else:
            return None          

In [42]:
class ReLu:
    """ReLu Activation function
    
    forward(x)
    x = the input to sigmoid function max(0,x)
        
    backward(gradient_l)
        gradient_l = the upstream gradient for backpropogation
        self.gradient = the local gradient of the Sigmoid activation function
        
        return the local gradients * the upstream gradient
    """
    def __init__(self):
        self.gradient = None
        self.x = None
    
    def forward(self,x):
        self.x = x
        if x > 0:
            return x
        else:
            return 0
    
    def backward(self,gradient_l=1):
        if self.x is not None:
            if self.x > 0:
                self.gradient = gradient_l
                return gradient_l
            else:
                self.gradient = 0
                return 0    
        else:
            return None

In [43]:
class Sigmoid:
    """Sigmoid Activation function
    
    forward(x)
        x = the input to sigmoid function f(x) = 1 / (1+exp(-x))
        
    backward(gradient_l)
        gradient_l = the upstream gradient for backpropogation
        self.gradient = the local gradient of the Sigmoid activation function
        
        return the local gradients * the upstream gradient
    """
    def __init__(self):
        self.gradient=None
        self.x = None
        
    def forward(self,x):
        self.x = x
        return 1/(1+np.exp(-x))
    
    def backward(self,gradient_l=1):
        if self.x is not None:
            self.gradient = 1/(1+np.exp(-self.x))*(1-1/(1+np.exp(-self.x)))*gradient_l
            return self.gradient
        else:
            return None

In [44]:
class BCE:
    """Binary Cross Entropy function
    
    forward(y_hat,y)
        y_hat = the predicted classification probability 0-1
        y = the training example actual label 0 or 1
        binary cross entropy = -(y*ln(y_hat) + (1-y)*ln(1-y_hat))
        
    backward(gradient_l)
        gradient_l = the upstream gradient for backpropogation
        self.gradient = the local gradient of the ReLu activation function
        
        return the local gradients * the upstream gradient
    """
    def __init__(self):
        self.gradient = None
        self.y = None
        self.y_hat = None
    
    def forward(self,y_hat,y):
        self.y_hat = y_hat
        self.y = y
        return -(y*np.log(max(sys.float_info.epsilon,y_hat-sys.float_info.epsilon)) + \
                 (1-y)*np.log(1-max(sys.float_info.epsilon,y_hat-sys.float_info.epsilon)))
    
    def backward(self,gradient_l=1):
        if self.y is not None and self.y_hat is not None:
            self.gradient = -self.y*1/(max(sys.float_info.epsilon,self.y_hat)) + \
                              1/(1-max(self.y_hat,sys.float_info.epsilon))*(1-self.y)
            return self.gradient
        else:
            return None
        
    def __str__(self):
        return "loss"

In [45]:
class neural_layer:
    """Neural Network layer
    
    __init__(input_,output_,bias=True)
        input_ = the number of features in the training example
        output_ = the number of neurons to use in the neural network layer
        bias = True if a bias term should be included for w*x+b 
               False if no bias term should be included for w*x+b
               default=True
    
    forward(x):
        x = input to the neuron for W*x + B
        
    backward(gradient_l):
        gradient_l = the upstream gradient for backpropogation
        gradients has gradient_x and gradient_w for each neuron in the neural network layer
        
        return the local gradients * the upstream gradient
    """
    def __init__(self,input_= 1,output_= 1,bias=True):
        self.output_size = output_
        self.input_size = input_
        self.neurons = [Neuron(input_,bias) for _ in range(output_)]
        
    def forward(self,x):
        outputs = [neuron.forward(x) for neuron in self.neurons]
        return np.concatenate(outputs,1)
    
    def backward(self,gradient_l=None):
        if gradient_l is None or None in gradient_l:
            return None
        
        gradients = []
        gradient_l = gradient_l.squeeze(0)       
        
        for i,neuron in enumerate(self.neurons):
            gradients.append(neuron.backward(gradient_l[i]))
            
        return gradients
    
    def __str__(self):
        return "neuron"

In [46]:
class action_layer:
    """Activation layer
    
    __init__(output_,a_type="ReLu")
        output_ = the number of activation units to use in the neural network layer
        a_type = the type of activation unit you want to use: "ReLu" or "Sigmoid"
    
    forward(x):
        x = input to the activation layer
        return the activation evaluation
        
    backward(gradient_l):
        gradient_l = the upstream gradient for backpropogation
        gradients has the local gradients for each activation unit
        
        return the local gradients * the upstream gradient
    """
    def __init__(self,output_= 1,a_type="ReLu"):
        
        self.output_size = output_
        
        if a_type == "ReLu":
            self.activations = [ReLu() for _ in range(output_)]
        if a_type == "Sigmoid":
            self.activations = [Sigmoid() for _ in range(output_)]
        
    def forward(self,x):
        output = []
        x = x.squeeze(0)
        for i,activation in enumerate(self.activations):
            output.append(activation.forward(x[i]))
        return np.expand_dims(output,0)
    
    def backward(self,gradient_l=None):
        if gradient_l is None or None in gradient_l:
            return None
        
        gradients = []
        gradient_l = gradient_l.squeeze(1)     
        
        for i,activation in enumerate(self.activations):
            gradients.append(activation.backward(gradient_l[i]))
            
        return np.expand_dims(gradients,0)
    
    def __str__(self):
        return "activation"

In [47]:
class neural_network:
    """Neural Network
    
    __init__(model=[])
        model = a list [] of all the neural network layer objects and activation layer objects. The forward and backward execution
                is sequential in regards to the order of the list passed in to model
    
    forward(x)
        x = the input training example to the neural network which will pass through all the layers of neural network and activation
            layers to classify or regress
    
    backward(gradient_l)
        gradient_l = the upstream gradient from the loss function to the neural network

        return all the gradients calculated throughout all the layers  
        
    model_weights()
        return a list with all the model weights for each neural network layer with neurons
    """
    def __init__(self,model=[]):
        self.model = model
        
    def forward(self,x):
        for obj in self.model:
            x = obj.forward(x)    
        return x
    
    def backward(self,gradient_l=None):
        gradients = []
        for obj in self.model[::-1]:
            if str(obj) == "activation":
                tup = ("activation",obj.backward(gradient_l))
                gradients.append(tup)
                gradient_l = tup[1]
            
            elif str(obj) == "neuron":
                tup = ("neuron",obj.backward(gradient_l))
                gradients.append(tup)
                
                if gradient_l is None or None in gradient_l:
                    gradient_l = None
                    continue
                
                gradient_l = 0
                for grad in tup[1]:
                    gradient_l += grad[1]  

        return gradients[::-1]
    
    def model_weights(self):
        weights = []
        layer_num = 0
        for obj in self.model:
            if str(obj) == 'neuron':
                layer_weights = []
                for neuron in obj.neurons:
                    layer_weights.append(neuron.weights)
                weights.append(('layer'+str(layer_num),layer_weights))
                layer_num += 1
        return weights
        

# Create Dataset (use the breast cancer dataset)
* Download and use the breast cancer dataset with sklearn
* standard normalize the data

In [55]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [56]:
X = data['data']
X = X - np.mean(X,0)
X = X / np.std(X,0)
Y = data['target']

# Attempt to do logistic regression proof of concept

In [58]:
neuron = Neuron(30)

In [59]:
activation = Sigmoid()

In [60]:
loss_fn = BCE()

In [61]:
alpha = 0.1

for i in range(100):
    mean_loss = []
    for x,y in zip(X,Y):
        x = np.expand_dims(x,0)

        # forward pass
        output = neuron.forward(x)
        a = activation.forward(output)
        
        loss = loss_fn.forward(a,y)
        mean_loss.append(loss)

        # backward pass
        loss_grad = loss_fn.backward()
        a_grad = activation.backward(loss_grad)
        _,weight_grad = neuron.backward(a_grad)

        # update model
        neuron.weights = neuron.weights - alpha*weight_grad
    print(i,np.mean(mean_loss))

0 [[0.16865002]]
1 [[0.08160309]]
2 [[0.06936675]]
3 [[0.06491617]]
4 [[0.06305983]]
5 [[0.06230607]]
6 [[0.06185728]]
7 [[0.06104828]]
8 [[0.06020596]]
9 [[0.05954941]]
10 [[0.05898587]]
11 [[0.05845267]]
12 [[0.0579555]]
13 [[0.05749291]]
14 [[0.05705958]]
15 [[0.05665435]]
16 [[0.05627402]]
17 [[0.05591503]]
18 [[0.05557539]]
19 [[0.05525354]]
20 [[0.05494786]]
21 [[0.05465686]]
22 [[0.05437928]]
23 [[0.05411406]]
24 [[0.05386022]]
25 [[0.0536169]]
26 [[0.05338332]]
27 [[0.0531588]]
28 [[0.05294272]]
29 [[0.05273452]]
30 [[0.0525337]]
31 [[0.05233979]]
32 [[0.05215238]]
33 [[0.05197108]]
34 [[0.05179554]]
35 [[0.05162543]]
36 [[0.05146045]]
37 [[0.05130033]]
38 [[0.05114481]]
39 [[0.05099364]]
40 [[0.05084661]]
41 [[0.0507035]]
42 [[0.05056411]]
43 [[0.05042827]]
44 [[0.05029581]]
45 [[0.05016657]]
46 [[0.05004038]]
47 [[0.04991712]]
48 [[0.04979665]]
49 [[0.04967884]]
50 [[0.04956358]]
51 [[0.04945075]]
52 [[0.04934025]]
53 [[0.04923198]]
54 [[0.04912585]]
55 [[0.04902177]]
56 [[0.

In [62]:
y_predictions = []
for x,y in zip(X,Y):
    x = np.expand_dims(x,0)

    # forward pass
    output = neuron.forward(x)
    a = activation.forward(output)
    
    y_predictions.append(a)

In [63]:
y_pred = np.round(np.array(y_predictions).squeeze())

In [64]:
accuracy = np.mean(y_pred == Y)
print(accuracy)

0.9876977152899824


# Neural Network proof of concept

In [65]:
model = [neural_layer(30,10),action_layer(10,"Sigmoid"),neural_layer(10,5),action_layer(5,"Sigmoid"),neural_layer(5,1),action_layer(a_type='Sigmoid')]
nn = neural_network(model)
loss_fn = BCE()

In [66]:
alpha = 0.1

for i in range(100):
    mean_loss = []
    for x,y in zip(X,Y):
        x = np.expand_dims(x,0)

        # forward pass
        output = nn.forward(x)
        loss = loss_fn.forward(output,y)
        mean_loss.append(loss)

        # backward pass
        loss_back = loss_fn.backward()
        nn_back = nn.backward(loss_back)

        for obj,grad in zip(nn.model,nn_back):
            if grad[0] == 'neuron':
                for n,grad_update in zip(obj.neurons,grad[1]):
                    n.weights = n.weights - alpha*grad_update[1]

    print(i,np.mean(mean_loss))

0 0.36971208512399917
1 0.18032312460218217
2 0.13214447475923818
3 0.11090338191791328
4 0.0984090582772083
5 0.09028666308952955
6 0.08486738104722792
7 0.08109836729528654
8 0.07833829509321606
9 0.07620718558843136
10 0.07447378024471511
11 0.07303207543348252
12 0.07182635868780293
13 0.07073027170993157
14 0.06959685013986586
15 0.06842278908653776
16 0.06722325210909366
17 0.0660004335166812
18 0.06478680597110413
19 0.06367662078719795
20 0.06271774608475764
21 0.06187110259658513
22 0.061096982424687564
23 0.06037415168216306
24 0.05969188936298923
25 0.059044836020668366
26 0.05843015203665773
27 0.05784590689349415
28 0.05729030756717159
29 0.05676144751615699
30 0.056257301893553456
31 0.05577579713860038
32 0.05531487807889012
33 0.05487255148088472
34 0.05444690873044744
35 0.05403613699514415
36 0.05363852730090537
37 0.05325248366786549
38 0.05287653237168449
39 0.05250932732154328
40 0.052149648341896504
41 0.051796392832556895
42 0.05144856468662236
43 0.0511052651170

In [67]:
y_predictions = []
for x,y in zip(X,Y):
    x = np.expand_dims(x,0)

    # forward pass
    output = nn.forward(x)
    
    y_predictions.append(output)

In [68]:
y_pred = np.round(np.array(y_predictions).squeeze())

In [69]:
accuracy = np.mean(y_pred == Y)
print(accuracy)

0.9894551845342706


In [70]:
nn.model_weights()

[('layer0',
  [array([[ 0.19286178],
          [-0.30298063],
          [-0.02019768],
          [-0.50455462],
          [ 0.0611002 ],
          [ 0.76532093],
          [ 0.33755652],
          [-1.00696302],
          [ 0.13815685],
          [ 0.38612644],
          [-0.88094713],
          [-0.89439378],
          [-0.70314072],
          [-0.46138417],
          [-0.07231421],
          [ 0.79833636],
          [-0.24130275],
          [ 0.20561182],
          [-0.26050891],
          [-0.16832721],
          [-0.82570205],
          [-1.42379458],
          [ 0.75051402],
          [-0.00428219],
          [-0.01661449],
          [-0.37045314],
          [-0.57274244],
          [-0.64491736],
          [-0.50055881],
          [ 0.73570731],
          [ 0.18612675]]),
   array([[ 0.83852024],
          [-0.37788092],
          [-0.60634691],
          [ 0.75953214],
          [ 0.19792922],
          [-0.24007013],
          [-1.18534436],
          [-0.81176259],
          [